<h1 style="color: red; text-align: center">Schema creation & data integration - Example</h1>
<hr width: 5px>
<h3>❖ Authors:</h3>
<p>TATI Mohammed<br>Ait Ouakour Said</p>
<p style="text-align: right;"><b>16-MAR | <span style="color: green;">2023</span></b></p>

## Required package

In [1]:
# Installing the connector
!pip install mysql-connector-python

In [2]:
# Importing the required library
from mysql import connector

## Needed Functions

In [3]:
def create_db():
    
    # Making the connection (Make sure MySQL & Apache services are running)
    conn = connector.Connect(host="localhost",    # your host, usually localhost
                             user="root",         # your username
                             port="3306")         # port (3306 default)
    # Creating the cursor
    cur = conn.cursor()
    
    # Creating our database
    try:
        cur.execute("CREATE DATABASE club_db")
        cur.execute("commit")
        # Closing the connection to the default database
        conn.close()
    except connector.Error as e:
        return e
    
    # Connecting to the new database
    try:
        conn = connector.Connect(host="localhost",    # your host, usually localhost
                                 user="root",         # your username
                                 port="3306",         # port (3306 default)
                                 db="club_db")        # name of the database
        # Creating the cursor
        cur = conn.cursor()
        print("Connected to club_db")
    
        return conn, cur
    except connector.Error as e:
        return e

def create_table(conn, cur):
    # Table data
    table_name = input('Enter the name of the table: ')
    cols_types = input("Enter the columns (separeted with comma) with their data types: ")
    table_name = table_name.strip()
    # Creating the table
    try:
        cur.execute("CREATE TABLE IF NOT EXISTS "+table_name+" ("+cols_types+");")
        conn.commit()
        
        return "Table "+table_name+" created!!"
    except connector.Error as e:
        return e

## Connecting to the database

In [4]:
# Connecting the club_db database
conn, cur = create_db()

Connected to club_db


## Schema creation

+ Let's create this database

<img src="rsc/ER.jpg" width="400">

+ <b>Note</b>:
    - When you are creating a database always start with the tables with no constraints (Foreign keys). 

<img src="rsc/MySQL-Data-Types.jpg" width="400">

### User table:

In [5]:
#cols: user_id int PRIMARY KEY, username varchar(50), email varchar(100), password varchar(30)
create_table(conn, cur)

Enter the name of the table: user
Enter the columns (separeted with comma) with their data types:  user_id int PRIMARY KEY, username varchar(50), email varchar(100), password varchar(30)


'Table user created!!'

### Product table:

In [6]:
#cols: product_id int PRIMARY KEY, product_name varchar(50), price double
create_table(conn, cur)

Enter the name of the table: product
Enter the columns (separeted with comma) with their data types: product_id int PRIMARY KEY, product_name varchar(50), price double


'Table product created!!'

### Order table:

In [7]:
#cols: order_id int PRIMARY KEY, product_id int, user_id int, quantity int, order_date date,
# constraints: FOREIGN KEY (user_id) REFERENCES user(user_id), FOREIGN KEY (product_id) REFERENCES product(product_id)
create_table(conn, cur)

Enter the name of the table: orders
Enter the columns (separeted with comma) with their data types: order_id int PRIMARY KEY, product_id int, user_id int, quantity int, order_date date,FOREIGN KEY (user_id) REFERENCES user(user_id), FOREIGN KEY (product_id) REFERENCES product(product_id)


'Table orders created!!'

## Data integration

In [8]:
# to manipulate the data
import pandas as pd

In [9]:
user = pd.read_csv('data/user.csv')
user

,user_id,username,email,password
0,1,johndoe,johndoe@example.com,password123
1,2,janesmith,janesmith@example.com,password456
2,3,bobwilson,bobwilson@example.com,password789


In [10]:
product = pd.read_csv('data/product.csv')
product

,product_id,product_name,price
0,1,Apple iPhone 12,799.52
1,2,Samsung Galaxy S21,799.96
2,3,Google Pixel 5,699.99


In [11]:
order = pd.read_csv('data/order.csv')
order

,order_id,product_id,user_id,quantity,order_date
0,1,2,1,1,2022-01-01
1,2,3,2,5,2022-01-02
2,3,1,1,15,2022-01-03


In [12]:
# Query to insert the data into user table
into_user = "INSERT INTO user(user_id, username, email, password) VALUES(%s, %s, %s, %s)"

# Query to insert the data into product table
into_product = "INSERT INTO product(product_id, product_name, price) VALUES(%s, %s, %s)"

# Query to insert the data into orders table
into_orders = "INSERT INTO orders(order_id, product_id, user_id, quantity, order_date) VALUES(%s, %s, %s, %s, %s)"

### Iterrows method

In [13]:
for i, row in product.iterrows():
    print(i, list(row))

0 [1, 'Apple iPhone 12', 799.52]
1 [2, 'Samsung Galaxy S21', 799.96]
2 [3, 'Google Pixel 5', 699.99]


### Inserting the product data

In [14]:
for i, row in product.iterrows():
    cur.execute(into_product, list(row))

# Saving
conn.commit()

### Inserting the user data

In [15]:
for i, row in user.iterrows():
    cur.execute(into_user, list(row))

# Saving
conn.commit()

### Inserting the orders data

In [16]:
for i, row in order.iterrows():
    cur.execute(into_orders, list(row))

# Saving
conn.commit()

## Closing the connection

In [17]:
conn.close()

<hr>
<p style="text-align: right">Schema creation & data integration - Example | <span style="color: green">Successfuly Completed</span></p>